## Investigate asyncio for parallel processing to reduce compute time

In [ ]:
# General imports
import os
import numpy as np
import matplotlib.pyplot as plt

# Segmentation
from astropy.io import fits
from photutils.segmentation import SourceFinder, make_2dgaussian_kernel, SourceCatalog
from photutils.background import Background2D, MedianBackground
from astropy.convolution import convolve

# grizli
from grizli.model import GrismFLT
import grizli

os.chdir("/Users/keith/astr/research_astr/summer-roman-project/all_chips/fits_scripts")

from roman_fits_utils import fix_hdr, rotate_img
from roman_seg_map import produce_seg_map


os.chdir("/Users/keith/astr/research_astr/all_chip_fits")

In [ ]:
# Functions for dynamic filenaming
file = lambda ii: "GRS_FOV0_roll0_dx0_dy0_SCA%i_direct_final.fits" %ii
newfile = lambda ii: "./ready_fits/rotated_img_%i.fits" %ii
seg_file = lambda ii: "seg_%i.fits" %ii

# Set variables and initiate sims dictionary
pad = 100
det_sizes = {"det1": 77,
            "det2": 80,
            "det3": 88,
            "det4": 180,
            "det5": 195,
            "det6": 209,
            "det7": 279,
            "det8": 302,
            "det9": 328,
            "det10": 98,
            "det11": 101,
            "det12": 110,
            "det13": 203,
            "det14": 217,
            "det15": 232,
            "det16": 303,
            "det17": 326,
            "det18": 353
            }

In [ ]:
# Prepare fits
def modify_and_rotate(ii):
    filename = file(ii)
    modified_hdr_name = "modified_hdr_%i.fits" %ii
    rotated_img_name = newfile(ii)

    hdr_dict = {"INSTRUME": "0;ROMAN",
                "FILTER":   "0;det%i" %ii,
                "CONFFILE": "1;/Users/keith/astr/research_astr/FOV0/Roman.det%i.07242020.conf" %ii}

    fix_hdr(filename, modified_hdr_name, hdr_dict)

    rotate_img(modified_hdr_name, rotated_img_name)

    print("Fits %i prepared" %ii)

    return None

In [ ]:
# Produce segmentation map
def segmentation_map(ii):
    filename = newfile(ii)
    seg_filename = seg_file(ii)

    produce_seg_map(filename, seg_filename)

    print("Seg map %i made" %ii)

    return None

In [ ]:
# Produce model
def run_sim(ii):
    size = det_sizes["det%i" %ii]
    foo = GrismFLT(direct_file=newfile(ii), seg_file=seg_file(ii), pad=pad)

    print("Model %i instantiated" %ii)

    foo.compute_full_model(mag_limit=28, compute_size=False, size=size)
    
    print("Model %i completed" %ii)

    return None

In [ ]:
# Run all functions
def run_full_sim(ii):
    modify_and_rotate(ii)
    segmentation_map(ii)
    foo = run_sim(ii)

    return foo

In [ ]:
# for ii in range(1, 19):
#     header = fits.open(file(ii))[0].header
#     fits.writeto("full_model_%i.fits" %ii, sims["%i_sim" %ii].model[pad:-pad, pad:-pad], header=header, overwrite=True)

In [1]:
import multiprocess as mp
from multiprocess.pool import Pool

In [2]:
from sim_full_mp import run_full_sim

In [3]:
def main():

    mp.set_start_method("spawn")

    with Pool(processes=3) as pool:
        
        sim = {}
        for ii, result in enumerate(pool.map(run_full_sim, range(1, 19), chunksize=6)):
            sim["%i"%ii] = result.model[pad:-pad, pad:-pad] / np.max(result.model)

    ax = ["ax%i"%ii for ii in range(1, 19)]
    fig, ax = plt.subplots(6, 3)

    ii = 1

    for x in range(3, 6):
        for y in range(-2, 1):
            y = -y
            ax[y, x].imshow(sim[str(ii)], vmin=0.01, vmax=0.15, cmap='hot')
            ax[y, x].set_title(str(ii))
            ii += 1

    for x in range(-2, 1):
        x = -x
        for y in range(-2, 1):
            y = -y
            ax[y, x].imshow(sim[str(ii)], vmin=0.01, vmax=0.15, cmap='hot')
            ax[y, x].set_title(str(ii))
            ii += 1

    return None

In [4]:
main()

Fits 1 prepared
Fits 4 prepared
Fits 13 prepared
Fits 7 prepared
Fits 16 prepared
Fits 10 prepared


Deblending:  92%|█████████▏| 5402/5878 [00:03<00:00, 1240.74it/s]

Seg map 1 made




Deblending:  89%|████████▉ | 5230/5863 [00:03<00:00, 1538.96it/s]

Image cutout: x=slice(0, 4088, None), y=slice(0, 4088, None) [Out of range]
./ready_fits/rotated_img_1.fits / Pad ref HDU with 464 pixels
Seg map 4 made
./ready_fits/rotated_img_1.fits / blot segmentation seg_1.fits[0]




...

Model 7 instantiated
Model 16 instantiated
Model 10 instantiated


Deblending: 100%|██████████| 5794/5794 [00:06<00:00, 861.88it/s] 


Seg map 13 made


Image cutout: x=slice(0, 4088, None), y=slice(0, 4088, None) [Out of range]
./ready_fits/rotated_img_13.fits / Pad ref HDU with 464 pixels
./ready_fits/rotated_img_13.fits / blot segmentation seg_13.fits[0]
Using default C-based coordinate transformation...
Using default C-based coordinate transformation...
Model 13 instantiated
ID 1.0 not found in segmentation image
ID 2.0 not found in segmentation image
ID 3.0 not found in segmentation image
ID 4.0 not found in segmentation image
ID 5.0 not found in segmentation image
ID 6.0 not found in segmentation image
ID 7.0 not found in segmentation image
ID 8.0 not found in segmentation image
ID 9.0 not found in segmentation image
ID 10.0 not found in segmentation image
ID 11.0 not found in segmentation image
ID 12.0 not found in segmentation image
ID 13.0 not found in segmentation image
ID 14.0 not found in segmentation image
ID 15.0 not found in segmentation image
ID 16.0 not found in segmentation image
ID 17.0 not found in

/Users/keith/miniconda3/envs/grizli_env/lib/python3.12/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 2 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/keith/miniconda3/envs/grizli_env/lib/python3.12/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 2 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/keith/miniconda3/envs/grizli_env/lib/python3.12/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 2 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/keith/miniconda3/envs/grizli_env/lib/python3.12/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 2 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: 

KeyboardInterrupt: 